In [58]:
import jupyros as jr # optional - for jupyter dev only
import ipyvolume as ipv # for pwointcloud visualization

In [59]:
import cv2
import image_geometry as img_geo
import matplotlib.pyplot as plt
import numpy as np
import os
import pcl
import pickle
import rosbag
import rospy
import tf

from pcl_helper import *
from cv_bridge import CvBridge, CvBridgeError
from geometry_msgs.msg import PoseStamped
from mpl_toolkits.mplot3d import Axes3D
from sensor_msgs.msg import Image, CameraInfo
from sklearn.decomposition import PCA
from __future__ import print_function

In [60]:
%matplotlib inline

### get an image
from `bag` or `publisher`. provides an `rgb` image and an `mm` depthimage.

In [61]:
bag = rosbag.Bag('../data/new_bin_with_screws2.bag', "r")
bridge = CvBridge()

In [62]:
topics = ['/head_camera/rgb/image_raw', '/head_camera/depth_registered/image', '/head_camera/depth_registered/points']
for topic, msg, _ in bag.read_messages(topics=topics):
    if topic==topics[0]:
        rgb_img = bridge.imgmsg_to_cv2(msg, desired_encoding='passthrough')
    elif topic==topics[1]:
        depth_img = bridge.imgmsg_to_cv2(msg, desired_encoding='passthrough')
    else:
        cloud = ros_to_pcl(msg)
        pcl_msg = msg

### mask green
to get bin outline

In [ ]:
real_green, green_benchmark = None, 'green.pkl'
if green_benchmark not in os.listdir('.'):
    with open(green_benchmark, 'w') as f:
        pickle.dump(green_benchmark, f)
with open(green_benchmark, 'r') as f:
    real_green = pickle.load(f)

In [ ]:
BIN_GREEN = np.mean(real_green, axis=0, dtype=int)
STDEV = 2.5*np.std(real_green, axis=0).astype(int)
ALPHA = 1.3

In [ ]:
hc_img = np.clip(rgb_img * ALPHA, 0, 255).astype(np.uint8)
mask = cv2.inRange(hc_img, BIN_GREEN-STDEV, BIN_GREEN+STDEV)
res = cv2.bitwise_and(hc_img, hc_img, mask=mask)

### filter only bolts

In [ ]:
plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2GRAY), cmap='gray')

In [ ]:
gray_img = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
_, contours, hierarchy = cv2.findContours(gray_img, 1, 2)

find convex hull of green bin


In [ ]:
max_contour = contours[np.argmax([cv2.contourArea(c) for c in contours])]
EPSILON = 0.1*cv2.arcLength(max_contour, True)
hull = cv2.convexHull(cv2.approxPolyDP(max_contour, EPSILON, True))

In [ ]:
res2 = res.copy(); plt.imshow(cv2.drawContours(res2, [max_contour], -1, (0,255,0), 3))

a less dumb approach: RANSAC away flat planes in depth

### find dark blobs

In [ ]:
HEIGHT_RANGE = int(0.3 * (np.max(hull[:,:,1]) - np.min(hull[:,:,1])))
WIDTH_RANGE = int(1.8 * HEIGHT_RANGE)

In [ ]:
cx, cy = np.mean(hull, axis=0)[0].astype(int)
roi_origin = (cy-HEIGHT_RANGE), (cx-WIDTH_RANGE)
roi = res[cy-HEIGHT_RANGE:cy+HEIGHT_RANGE,\
   cx-WIDTH_RANGE:cx+WIDTH_RANGE,:]
roi[roi < np.array([50, 25, 50])] = 0 # threshold

In [ ]:
gray_img = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
_, contours, hierarchy = cv2.findContours(gray_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
areas = np.array([cv2.contourArea(c) for c in contours])
mean_area = np.mean(areas)
part_area = 0.6*mean_area
MIN_AREA, MAX_AREA = mean_area-part_area, mean_area+2*part_area

In [ ]:
candidates = np.where((areas>MIN_AREA) & (areas<MAX_AREA))[0]

In [ ]:
plt.imshow(roi)

### singulate a bolt
score = w * (dist from top of image + flatness of bounding box)

(as measures of greatest operating volume for end-effector & top-down surface area of bolt)

In [ ]:
def bounding_box(contour):
    rect = cv2.minAreaRect(contour)
    box = np.int0(cv2.boxPoints(rect))
    x, y = np.mean(box[:,0]), np.mean(box[:,1])
    _, _, angle = cv2.fitEllipse(contour)
    return int(x), int(y), angle, box

In [ ]:
def score(x, y, angle):
    dist_from_normal = abs((90.0-angle)/180.0)
    return y/480.0 + (1-dist_from_normal)

In [ ]:
entities, meta = [], []
fig, ax = plt.subplots(figsize=(30,10), nrows=2, ncols=3)
for i, candidate in enumerate(candidates):
    x, y, angle, box = bounding_box(contours[candidate])
    entities.append(box)
    meta.append((x, y, angle))
    res2 = roi.copy()
    ax.flat[i].imshow(cv2.drawContours(res2, [entities[-1]], 0, (0, 255, 0), 2))

`(x, y, z)` give 3D pose wrt camera frame

In [ ]:
K_TABLE_HEIGHT = np.inf # unknown

In [ ]:
i_best = np.argmax([score(x, y, angle) for (x, y, angle) in meta])
x = int(meta[i_best][1] + roi_origin[0])
y = int(meta[i_best][0] + roi_origin[1])
z = min(depth_img[x, y], K_TABLE_HEIGHT)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5), nrows=1, ncols=2)
ax[0].imshow(rgb_img[x-50:x+50, y-50:y+50])
ax[1].imshow(depth_img[x-50:x+50, y-50:y+50], cmap='gray')
for axes in ax:
    axes.plot(50, 50, 'o', color='r')
plt.show()

In [ ]:
x, y, z

you must run `roscore` first

publish a `tf` to get the correct transforms

only run `init_node` once

In [ ]:
rospy.init_node('test')
while not rospy.Time.now():
    pass
import time; time.sleep(5) # hotfix to let TF buffer fill
tfl = tf.TransformListener()

TODO make `camera` a class field, so `xyz_to_pose` can grab it

In [ ]:
camera = img_geo.PinholeCameraModel() 
camera.fromCameraInfo(rospy.wait_for_message('/head_camera/depth/camera_info', CameraInfo))
frame_x, frame_y, _ = camera.projectPixelTo3dRay((x, y))
frame_z = z/1000.0

In [ ]:
# assumes a top-down grasp
def xyz_to_pose(x, y, z, yaw=np.radians(90), src_frame='/base_link', dst_frame='/head_camera_rgb_optical_frame'):
    pose = PoseStamped()
    pose.header.frame_id = dst_frame
    pose.header.stamp = tfl.getLatestCommonTime(src_frame, dst_frame) #rospy.Time.now()
    pose.pose.position.x = x
    pose.pose.position.y = y
    pose.pose.position.z = z if z<=1000.0 else z/1000.0 # max depth range 1 meter
    pose.pose.orientation.w = 1.0
    tfl.waitForTransform(camera.tfFrame(), src_frame, tfl.getLatestCommonTime(src_frame, dst_frame), rospy.Duration(5.0))
    target = tfl.transformPose(src_frame, pose)
    
    # make top-down
    q = tf.transformations.quaternion_from_euler(np.radians(180), np.radians(90), yaw)
    target.pose.orientation.x = q[0]
    target.pose.orientation.y = q[1]
    target.pose.orientation.z = q[2]
    target.pose.orientation.w = q[3]
    return target

In [ ]:
frame_x, frame_y, frame_z

In [ ]:
target = xyz_to_pose(frame_x, frame_y, frame_z, yaw=np.radians(90))

In [ ]:
target

### pcl

In [63]:
def view_pcl(X, color='red'):
    N = X.T
    if type(X)!='numpy.ndarray':
        N = np.asarray(X).T
    ipv.figure()
    ipv.quickscatter(N[0,], N[1,], N[2,], size=1, marker='point_2d', color=color)
    ipv.xyzlim(-1.0, 1.5)
    ipv.show()

In [64]:
theta = np.radians(56)
T = np.array([
    [1, 0, 0, 0],
    [0, np.cos(theta), -np.sin(theta), 0],
    [0, np.sin(theta), np.cos(theta), 0],
    [0, 0, 0, 1]
])
X = np.dot(cloud.to_array(), T)
X = XYZRGB_to_XYZ(X)
#view_pcl(np.asarray(X))

filter out most of table

In [65]:
TABLE_HEIGHT = (0.0, 0.7) # 0.6
TABLE_DEPTH = (0.0, 1.25)

In [66]:
filter_axis, passthrough = 'y', X.make_passthrough_filter()
passthrough.set_filter_field_name('y')
passthrough.set_filter_limits(*TABLE_HEIGHT)
passthrough = passthrough.filter().make_passthrough_filter()
passthrough.set_filter_field_name('z')
passthrough.set_filter_limits(*TABLE_DEPTH)
X = passthrough.filter()

In [67]:
view_pcl(X.to_array())

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMC4wLCAyLjApLCBxdWF0ZXJuaW9uPSgwLjAsIDAuMCwgMC4wLCDigKY=


In [68]:
seg = X.make_segmenter()
seg.set_model_type(pcl.SACMODEL_PLANE)
seg.set_method_type(pcl.SAC_RANSAC)
max_distance = 0.01
seg.set_distance_threshold(max_distance)

# Extract inliers and outliers
inliers, coefficients = seg.segment()
table = X.extract(inliers, negative=False)
objects = X.extract(inliers, negative=True)

In [69]:
view_pcl(np.asarray(objects))

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMC4wLCAyLjApLCBxdWF0ZXJuaW9uPSgwLjAsIDAuMCwgMC4wLCDigKY=


### get ground truth (bolt) pointcloud

In [70]:
bolt_truth = pcl.load('../data/bolt.pcd')
voxels = bolt_truth.make_voxel_grid_filter()

In [71]:
LEAF_SIZE = 1.0
voxels.set_leaf_size(LEAF_SIZE, LEAF_SIZE, LEAF_SIZE)
bolt_truth = voxels.filter()

resize for our camera frame

In [72]:
TRUTH_SCALE = 1000.0

In [73]:
scaled = np.asarray(bolt_truth).T.copy()/TRUTH_SCALE
bolt_truth = pcl.PointCloud(scaled.T)
ipv.figure()
ipv.quickscatter(scaled[0,], scaled[1,], scaled[2,], size=1, marker='sphere', color='blue')
ipv.xyzlim(0,0.2)
ipv.show()

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMC4wLCAyLjApLCBxdWF0ZXJuaW9uPSgwLjAsIDAuMCwgMC4wLCDigKY=


### euclidean clustering
find big blobs

In [74]:
CLUSTER_SIZE = 0.01
MIN_CLUSTER, MAX_CLUSTER = 400, 1000

In [75]:
tree = objects.make_kdtree()
ec = objects.make_EuclideanClusterExtraction()
ec.set_ClusterTolerance(CLUSTER_SIZE)
ec.set_MinClusterSize(MIN_CLUSTER)
ec.set_MaxClusterSize(MAX_CLUSTER)
ec.set_SearchMethod(tree)
cluster_indices = ec.Extract()

In [76]:
clusters = []
for j, indices in enumerate(cluster_indices):
    points = np.zeros((len(indices), 3), dtype=np.float32)

    for i, index in enumerate(indices):
        points[i][0] = objects[index][0]
        points[i][1] = objects[index][1]
        points[i][2] = objects[index][2]

    cluster = pcl.PointCloud()
    cluster.from_array(points)
    clusters.append(cluster)

In [77]:
ipv.figure()
N = np.asarray(objects).T
ipv.scatter(N[0,], N[1,], N[2,], size=2, marker='point_2d', color='red')
ipv.xyzlim(-0.4, 0.8)

for cluster in clusters:
    N = np.asarray(cluster).T
    ipv.scatter(N[0,], N[1,], N[2,], size=3, marker='point_2d', color='blue')
ipv.show()

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMC4wLCAyLjApLCBxdWF0ZXJuaW9uPSgwLjAsIDAuMCwgMC4wLCDigKY=


### ICP
minimize dissimilarity b/w ground truth bolt and any blob,
return transform

In [78]:
target = clusters[5]

In [79]:
target = pcl.PointCloud(np.array(target).astype(np.float32))
icp = bolt_truth.make_IterativeClosestPoint()
converged, transf, estimate, fitness = icp.icp(bolt_truth, target)

/home/neil/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Non length-one string passed in for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  This is separate from the ipykernel package so we can avoid doing imports until


transform back to original depth camera frame

In [81]:
orig_cluster = np.dot(target, np.linalg.inv(T[:3,:3]))
np.mean(orig_cluster, axis=0)

array([-0.04175097,  0.05268445,  0.75619031])

all transformations & registration

In [82]:
transf[:3,:3]

array([[ 0.23753771, -0.02622684,  0.9710252 ],
       [-0.5075893 ,  0.84895104,  0.14709911],
       [-0.8282101 , -0.5278231 ,  0.1883448 ]], dtype=float32)

### ICP fails
although $ T^{truth}_{target} $ is accurate, noise causes a match that may result in a failing grasp. we should ignore priors from the existing bolt model

In [83]:
ipv.figure()
#N = np.asarray(clusters[5]).T
#ipv.scatter(N[0,], N[1,], N[2,], size=1, marker='sphere', color='blue')
#N = np.asarray(objects).T
#ipv.scatter(N[0,], N[1,], N[2,], size=2, marker='point_2d', color='yellow')

N = np.asarray(orig_cluster).T
ipv.scatter(N[0,], N[1,], N[2,], size=1, marker='sphere', color='blue')
N = np.asarray(cloud).T
ipv.scatter(N[0,], N[1,], N[2,], size=2, marker='point_2d', color='red')

N = np.asarray(bolt_truth).T
ipv.scatter(N[0,], N[1,], N[2,], size=1, marker='sphere', color='yellow')
N = np.dot(np.asarray(bolt_truth), transf[:3,:3]).T
ipv.scatter(N[0,], N[1,], N[2,], size=1, marker='sphere', color='blue')

#N = np.asarray(estimate).T
#ipv.scatter(N[0,], N[1,], N[2,], size=3, marker='point_2d', color='blue')
ipv.xyzlim(-0.4, 0.8)
ipv.show()

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMC4wLCAyLjApLCBxdWF0ZXJuaW9uPSgwLjAsIDAuMCwgMC4wLCDigKY=


In [89]:
np.asarray(target)

array([[-0.06190419,  0.6482394 ,  0.39089406],
       [-0.06046522,  0.6482394 ,  0.39089406],
       [-0.05902625,  0.6482394 ,  0.39089406],
       ...,
       [-0.0575111 ,  0.6626961 ,  0.3676728 ],
       [-0.05607403,  0.6626961 ,  0.3676728 ],
       [-0.05470932,  0.66357386,  0.3681598 ]], dtype=float32)

In [90]:
pca = PCA(n_components=3, copy=True)
pca.fit(np.asarray(target))
pca.mean_

array([-0.04175099,  0.6563714 ,  0.3791789 ], dtype=float32)

In [85]:
def draw_vector(v0, v1, v2, ax=None):
    ax = ax or plt.figure().add_subplot(111, projection='3d')
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1[:2], v0[:2], arrowprops=arrowprops)